In [1]:
import pandas as pd
import json

In [2]:
LOG_FILE = "game_overs_2014_05_21.txt"
OUTPUT_FILE= "game_overs_compact.csv"
logs = []
with open(LOG_FILE,'r') as file:
   for log in file:
       row = json.loads(log)
       logs.append(row)
df = pd.DataFrame(logs)

# Slicing

In [3]:
# work with only the version 1.0 of the application.
df['version'] = df['ver'].astype(str)
df =  df[df['version'] == "1.0"]

In [4]:
# work with only the first 50 levels
df = df[df['level_number'] <= 50]

# remove rows with end_reason equal to "quit"
df = df[~df['end_reason'].isin(['quit'])]

In [5]:
# remove columns that do not apply to the current version or that has debug/ dev data
"""df = df.drop(['application', 'msg', 'type', 'ver', 'headstart', 'levelpack',
              'n_evil_masks', 'n_fountains', 'n_freezers','n_frozen_spikyplants', 
              'n_trampoline_free','n_pineapple_value', 'n_pineapples',
              'bonus_bb2.tigerstrike', 'collected_bb2.tigerstrike',
              'bonus_bb2.snakeoil', 'collected_bb2.snakeoil', 
              'n_chilis', 'n_tiger_strikes', 'n_tigers',
              'happening_before_death','level'], axis=1)"""
df =df[['device_id', 'ts', 'level_number', 
        'end_reason', 'lives_left','n_bananavalue', 'n_distance',
        'n_specialcollectibles', 'n_swings', 'pathtrace',
        'stars','swings_left', 'time_used', 'total_duration','score']]

# Cleaning

In [6]:
df = df.drop_duplicates()
df.describe()

,ts,level_number,lives_left,n_bananavalue,n_distance,n_specialcollectibles,n_swings,stars,swings_left,time_used,total_duration,score
count,7.825610e+05,782561.000000,7.825610e+05,782561.000000,782561.000000,782560.000000,782560.000000,782561.000000,782561.000000,782561.000000,782561.000000,782560.000000
mean,1.400738e+09,12.461257,6.860794e+04,79.075409,283.841021,0.035050,9.229953,1.291405,0.512084,38.000148,45.359144,1314.369662
std,6.094903e+06,9.551357,1.213764e+07,141.405982,223.004633,0.387486,7.309185,1.435501,1.512776,29.732864,30.399311,1507.377595
min,1.049000e+03,1.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
25%,1.400643e+09,5.000000,2.000000e+00,0.000000,93.000000,0.000000,2.000000,0.000000,0.000000,10.000000,21.000000,100.000000
50%,1.400653e+09,10.000000,4.000000e+00,0.000000,232.000000,0.000000,8.000000,0.000000,0.000000,31.000000,38.000000,940.000000
75%,1.400660e+09,18.000000,5.000000e+00,115.000000,425.000000,0.000000,14.000000,3.000000,0.000000,61.000000,66.000000,1816.000000
max,2.095238e+09,50.000000,2.147484e+09,2818.000000,2118.000000,10.000000,60.000000,3.000000,21.000000,898.000000,2322.000000,28229.000000


In [7]:
# remove rows with negative distances
df = df[df['n_distance'] >= 0]

# remove outliers
df = df[df['lives_left'] <= 50]

# Transforming

In [9]:
# convert timestamp to datetime
df['date_time'] = pd.to_datetime(df['ts'], unit='s')

# remove rows with wrong date
df = df[df['date_time'].dt.date == pd.to_datetime('2014-05-21').date()]
df = df.drop(['ts'], axis=1)

In [10]:
df.to_parquet('game_overs_compact.parquet', engine='pyarrow', compression='zstd')